In [27]:
import requests
import defs
import pandas as pd


Creation a connection session to make an API request

In [3]:
session = requests.Session()

Define number of candles to get, Time Frame and URL

In [4]:
instrument = "EUR_USD"
count = 10
granularity = "H1"

In [5]:
url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
url

'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles'

Create a Dictionary to group parameters to pass the Response


In [6]:
params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
)
params

{'count': 10, 'granularity': 'H1', 'price': 'MBA'}

Make the response

In [7]:
response = session.get(url, params=params, headers=defs.SECURE_HEADER)

Confirm the response is successful (200 OK)

In [8]:
response.status_code

200

Review Response Body

In [9]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 7782,
   'time': '2022-01-27T14:00:00.000000000Z',
   'bid': {'o': '1.11485', 'h': '1.11615', 'l': '1.11448', 'c': '1.11603'},
   'mid': {'o': '1.11492', 'h': '1.11622', 'l': '1.11455', 'c': '1.11610'},
   'ask': {'o': '1.11499', 'h': '1.11629', 'l': '1.11462', 'c': '1.11618'}},
  {'complete': True,
   'volume': 8257,
   'time': '2022-01-27T15:00:00.000000000Z',
   'bid': {'o': '1.11605', 'h': '1.11609', 'l': '1.11308', 'c': '1.11487'},
   'mid': {'o': '1.11612', 'h': '1.11617', 'l': '1.11316', 'c': '1.11494'},
   'ask': {'o': '1.11620', 'h': '1.11625', 'l': '1.11323', 'c': '1.11500'}},
  {'complete': True,
   'volume': 5951,
   'time': '2022-01-27T16:00:00.000000000Z',
   'bid': {'o': '1.11485', 'h': '1.11611', 'l': '1.11404', 'c': '1.11416'},
   'mid': {'o': '1.11492', 'h': '1.11618', 'l': '1.11411', 'c': '1.11423'},
   'ask': {'o': '1.11499', 'h': '1.11626', 'l': '1.11418', 'c': '1.11430'}},

Store JSON Response

In [10]:
data = response.json()

In [11]:
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [14]:
len(data['candles'])

10

In [16]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [17]:
for price in prices:
    for oh in ohlc:
        print(f"{price}_{oh}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [21]:
data['candles'][0]

{'complete': True,
 'volume': 7782,
 'time': '2022-01-27T14:00:00.000000000Z',
 'bid': {'o': '1.11485', 'h': '1.11615', 'l': '1.11448', 'c': '1.11603'},
 'mid': {'o': '1.11492', 'h': '1.11622', 'l': '1.11455', 'c': '1.11610'},
 'ask': {'o': '1.11499', 'h': '1.11629', 'l': '1.11462', 'c': '1.11618'}}

In [22]:
data['candles'][0]['bid']['o']

'1.11485'

In [26]:
our_data = []
for candle in data['candles']:
    if candle['complete'] == False:
        continue
    new_dict ={}
    new_dict['time'] = candle['time']
    new_dict['volume'] = candle['volume']
    # print(new_dict)

    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"] = candle[price][oh]

    our_data.append(new_dict)
# print(our_data)
print(our_data[0])

{'time': '2022-01-27T14:00:00.000000000Z', 'volume': 7782, 'mid_o': '1.11492', 'mid_h': '1.11622', 'mid_l': '1.11455', 'mid_c': '1.11610', 'bid_o': '1.11485', 'bid_h': '1.11615', 'bid_l': '1.11448', 'bid_c': '1.11603', 'ask_o': '1.11499', 'ask_h': '1.11629', 'ask_l': '1.11462', 'ask_c': '1.11618'}


Now we make a new Data Frame from all our structured data of MID, BID, ASK x OHLC

In [28]:
candles_df = pd.DataFrame.from_dict(our_data)

In [29]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-01-27T14:00:00.000000000Z,7782,1.11492,1.11622,1.11455,1.11610,1.11485,1.11615,1.11448,1.11603,1.11499,1.11629,1.11462,1.11618
1,2022-01-27T15:00:00.000000000Z,8257,1.11612,1.11617,1.11316,1.11494,1.11605,1.11609,1.11308,1.11487,1.11620,1.11625,1.11323,1.11500
2,2022-01-27T16:00:00.000000000Z,5951,1.11492,1.11618,1.11411,1.11423,1.11485,1.11611,1.11404,1.11416,1.11499,1.11626,1.11418,1.11430
3,2022-01-27T17:00:00.000000000Z,3445,1.11424,1.11515,1.11366,1.11392,1.11417,1.11508,1.11359,1.11385,1.11431,1.11523,1.11372,1.11398
4,2022-01-27T18:00:00.000000000Z,2658,1.11390,1.11468,1.11356,1.11445,1.11383,1.11461,1.11349,1.11437,1.11397,1.11476,1.11364,1.11453
5,2022-01-27T19:00:00.000000000Z,2307,1.11446,1.11471,1.11388,1.11405,1.11439,1.11464,1.11380,1.11398,1.11454,1.11478,1.11394,1.11412
6,2022-01-27T20:00:00.000000000Z,1936,1.11404,1.11494,1.11400,1.11427,1.11397,1.11487,1.11393,1.11421,1.11411,1.11500,1.11408,1.11433
7,2022-01-27T21:00:00.000000000Z,468,1.11426,1.11453,1.11416,1.11441,1.11419,1.11446,1.11409,1.11433,1.11432,1.11461,1.11423,1.11449
8,2022-01-27T22:00:00.000000000Z,174,1.11473,1.11473,1.11392,1.11444,1.11423,1.11433,1.11360,1.11420,1.11523,1.11523,1.11419,1.11469


Save results to file so new updates just append to same file for future use with updated data

In [30]:
candles_df.to_pickle("EUR_USD_H1.pkl")

In [31]:
test_df = pd.read_pickle("EUR_USD_H1.pkl")
test_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-01-27T14:00:00.000000000Z,7782,1.11492,1.11622,1.11455,1.11610,1.11485,1.11615,1.11448,1.11603,1.11499,1.11629,1.11462,1.11618
1,2022-01-27T15:00:00.000000000Z,8257,1.11612,1.11617,1.11316,1.11494,1.11605,1.11609,1.11308,1.11487,1.11620,1.11625,1.11323,1.11500
2,2022-01-27T16:00:00.000000000Z,5951,1.11492,1.11618,1.11411,1.11423,1.11485,1.11611,1.11404,1.11416,1.11499,1.11626,1.11418,1.11430
3,2022-01-27T17:00:00.000000000Z,3445,1.11424,1.11515,1.11366,1.11392,1.11417,1.11508,1.11359,1.11385,1.11431,1.11523,1.11372,1.11398
4,2022-01-27T18:00:00.000000000Z,2658,1.11390,1.11468,1.11356,1.11445,1.11383,1.11461,1.11349,1.11437,1.11397,1.11476,1.11364,1.11453
5,2022-01-27T19:00:00.000000000Z,2307,1.11446,1.11471,1.11388,1.11405,1.11439,1.11464,1.11380,1.11398,1.11454,1.11478,1.11394,1.11412
6,2022-01-27T20:00:00.000000000Z,1936,1.11404,1.11494,1.11400,1.11427,1.11397,1.11487,1.11393,1.11421,1.11411,1.11500,1.11408,1.11433
7,2022-01-27T21:00:00.000000000Z,468,1.11426,1.11453,1.11416,1.11441,1.11419,1.11446,1.11409,1.11433,1.11432,1.11461,1.11423,1.11449
8,2022-01-27T22:00:00.000000000Z,174,1.11473,1.11473,1.11392,1.11444,1.11423,1.11433,1.11360,1.11420,1.11523,1.11523,1.11419,1.11469
